## RNN, LSTM, GRU in Pytorch

#### Import Library

In [1]:
import os
import time
import torch
from tqdm import tqdm

from torch import nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data.dataset import random_split

from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.data.functional import to_map_style_dataset
from torchtext.vocab import build_vocab_from_iterator, Vectors

In [2]:
config = {'pre_trained' : 'glove', # 'glove','fasttext', None 
          'max_length': 300,
          'batch_size': 64,
          'model_type': 'gru', # 'rnn', 'lstm', 'gru','avg_not_pad', None
          'emb_dim' : 300,
          'hidden_dim':128,
          'is_bidirectional':True,
          'epoch' : 15,
          'LR': 5
          }

#### Reading Data

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

# Tokenize & Vocab setup
tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='train')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), min_freq= 2, specials=["<unk>", "<pad>"])
vocab.set_default_index(vocab["<unk>"]) # This index will be returned when OOV token is queried.

num_class = 2
vocab_size = len(vocab)
idx_pad = vocab.get_stoi()['<pad>']

In [5]:
if config['pre_trained'] == 'glove':
    pretrained_vectors = Vectors(name = 'glove.6B.300d.txt', 
                                #  cache = '[my_path]',
                                 url = 'http://nlp.stanford.edu/data/glove.6B.zip')
    pretrained_emb = pretrained_vectors.get_vecs_by_tokens(vocab.get_itos(), lower_case_backup=True)

elif config['pre_trained'] == 'fasttext':
    pretrained_vectors = Vectors(name = 'wiki.simple.vec', 
                                #  cache = '[my_path]',
                                 url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.simple.vec')
    pretrained_emb = pretrained_vectors.get_vecs_by_tokens(vocab.get_itos(), lower_case_backup=True)
else:
    pass

In [6]:
# DataLoader Setup
text_pipeline = lambda x: vocab(tokenizer(x))[:config['max_length']]
label_pipeline = lambda x: {"neg":0, "pos":1}.get(x)

def collate_batch(batch):
    label_list, text_list = [], [] 
    for (_label, _text) in batch:
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        label_list.append(label_pipeline(_label))
    
    text_list = pad_sequence(text_list, batch_first= True, padding_value= idx_pad)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    return text_list.to(device), label_list.to(device)

train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_batch)

In [8]:
# Data Sample
print('---- Data Sample ----')
print('Input : ')
print(' '.join(vars(train_data.examples[1])['text']),'\n')
print('Label : ')
print(vars(train_data.examples[1])['label'])

---- Data Sample ----
Input : 
Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the str

#### Data Preprocessing

In [9]:
def PreProcessingText(input_sentence):
    input_sentence = input_sentence.lower() # 소문자화
    input_sentence = re.sub('<[^>]*>', repl= ' ', string = input_sentence) # "<br />" 처리
    input_sentence = re.sub('[!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~]', repl= ' ', string = input_sentence) # 특수문자 처리 ("'" 제외)
    input_sentence = re.sub('\s+', repl= ' ', string = input_sentence) # 연속된 띄어쓰기 처리
    if input_sentence:
        return input_sentence

#### Model Setup

In [7]:
# Model setup
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, num_class, **config):
        super(TextClassificationModel, self).__init__()
        self.model_type = config['model_type']
        self.pretrained = config['pre_trained']
        self.is_bidirectional = config['is_bidirectional']
        self.embed_dim = config['emb_dim']
        self.hidden_dim = config['hidden_dim']
        
        self.embedding = nn.Embedding(vocab_size, self.embed_dim,)
        if self.pretrained:
            self.embedding = nn.Embedding(vocab_size, self.embed_dim,).from_pretrained(pretrained_emb, freeze = False)

        if self.model_type is None:
            self.fc = nn.Linear(self.embed_dim, num_class)

        elif self.model_type == 'avg_not_pad':
            self.embedding = nn.EmbeddingBag(vocab_size, self.embed_dim, sparse=True, padding_idx = idx_pad)
            if self.pretrained:
                self.embedding = self.embedding.from_pretrained(pretrained_emb, freeze = False, sparse=True)
            self.fc = nn.Linear(self.embed_dim, num_class)

        elif self.model_type in ['rnn','lstm','gru']:
            if self.model_type == 'rnn':
                self.Recurrent = nn.RNN(input_size = self.embed_dim, hidden_size = self.hidden_dim, 
                                        bidirectional = self.is_bidirectional, batch_first = True)
            elif self.model_type == 'lstm':
                self.Recurrent = nn.LSTM(input_size = self.embed_dim, hidden_size = self.hidden_dim, 
                                         bidirectional = self.is_bidirectional, batch_first = True)
            else:
                self.Recurrent = nn.GRU(input_size = self.embed_dim, hidden_size = self.hidden_dim, 
                                        bidirectional = self.is_bidirectional, batch_first = True)

            last_input_dim = self.hidden_dim * 2 if self.is_bidirectional else self.hidden_dim 
            self.fc = nn.Linear(last_input_dim, num_class)

        else:
            raise NameError('Select model_type in [rnn, lstm, gru, avg_not_pad]')

        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        if self.pretrained:
            self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text):
        embedded = self.embedding(text)
        if self.model_type is None:
            embedded = torch.mean(embedded, dim=1)
            return self.fc(embedded)
        elif self.model_type == 'avg_not_pad':
            return self.fc(embedded)
        else:
            output, _ = self.Recurrent(embedded)
            last_output = output[:,-1,:]
            return self.fc(last_output)


model = TextClassificationModel(vocab_size, num_class, **config).to(device)
model

TextClassificationModel(
  (embedding): Embedding(51718, 300)
  (Recurrent): GRU(300, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)

In [ ]:
# Old version
class SentenceClassification(nn.Module):
    def __init__(self, **model_config):
        super(SentenceClassification, self).__init__()

        if model_config['emb_type'] == 'glove' or 'fasttext':
            # nn.Embedding을 통해 쉽게 임베딩 설정 가능
            self.emb = nn.Embedding(num_embeddings = model_config['vocab_size'],
                                    embedding_dim = model_config['emb_dim'],
                                    _weight = TEXT.vocab.vectors)
            # num_embeddings = Vocab Size, embedding_dim = 원하는 임베딩 차원을 설정 (Pre-trained Vector 사용 시, 차원과 일치 시켜야함)
            # _weight = Pre-trained Vector를 Embedding 행렬의 Initial Value로 설정, 모델 학습을 이대로 진행하면 해당 Embedding Layer의 행렬도 학습
            # nn.Embedding.from_pretrained(TEXT.vocab.vectors)은 학습 freeze 상태, 현재 옵션 아니라면 정규 분포에서 생성한 값을 Initial Value로 사용 
        else:
            self.emb = nn.Embedding(num_embeddings = model_config['vocab_size'],
                                    embedding_dim = model_config['emb_dim'])
        
        self.bidirectional = model_config['bidirectional']
        self.num_direction = 2 if model_config['bidirectional'] else 1
        self.model_type = model_config['model_type'] 

        # RNN, GRU, LSTM의 필수 옵션은 대부분 유사
        self.RNN = nn.RNN (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        # input_size : 입력받을 Data의 크기이므로, Embedding Dimension으로 설정
        # hidden_size : Hidden Layer의 Dimension을 설정
        # Dropout : Dropout 확률, bidirectional: 양방향 모델을 사용할 경우 설정, batch_first: Data의 제일 처음 Axis에 Batch_size가 오도록 설정
        
        self.LSTM= nn.LSTM(input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
        
        self.GRU = nn.GRU (input_size = model_config['emb_dim'],
                           hidden_size = model_config['hidden_dim'],
                           dropout = model_config['dropout'],
                           bidirectional = model_config['bidirectional'],
                           batch_first = model_config['batch_first'])
    
        self.fc = nn.Linear(model_config['hidden_dim'] * self.num_direction,
                            model_config['output_dim'])
        # 위 정보를 이용해 분류 문제룰 푸는 Task를 할 예정이므로, Class에 대한 Score를 생성하기 위해 FC layer 1개를 통과, 추가 시그모이드가 없는 이유는 추후 Loss Function에 포함돼 있기 때문
        
        self.drop = nn.Dropout(model_config['dropout'])

    def forward(self, x):
        
        emb = self.emb(x) 
        # emb : (Batch_Size, Max_Seq_Length, Emb_dim)

        if self.model_type == 'RNN':
            output, hidden = self.RNN(emb) 
        elif self.model_type == 'LSTM':
            output, (hidden, cell) = self.LSTM(emb)
        elif self.model_type == 'GRU':
            output, hidden = self.GRU(emb)
        else:
            raise NameError('Select model_type in [RNN, LSTM, GRU]')
        
        # output : (Batch_Size, Max_Seq_Length, Hidden_dim * num_direction) 
        # hidden : (num_direction, Batch_Size, Hidden_dim)
        # hidden의 경우, batch_first 옵션이 안먹는 문제가 있음
        
        last_output = output[:,-1,:]

        # last_output : (Batch_Size, Hidden_dim * num_direction)
        return self.fc(self.drop(last_output))

#### Training

In [8]:
# Training Setup
# Hyperparameters
EPOCHS = config['epoch']
LR = config['LR']  

total_accu = None

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()

    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (text, label) in tqdm(enumerate(train_dataloader)):
        # Training
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(f'| epoch {epoch:3d} | {idx:5d}/{len(train_dataloader):5d} batches | accuracy {total_acc/total_count:8.3f}')                             
            total_acc, total_count = 0, 0
            start_time = time.time()
        
    # Evaluation
    model.eval()
    total_acc, total_count = 0, 0
    with torch.no_grad():
        for idx, (text, label) in enumerate(valid_dataloader):
            predicted_label = model(text)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    accu_val =  total_acc/total_count  
     
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print(f'| end of epoch {epoch:3d} | time: {time.time() - epoch_start_time:5.2f}s | valid accuracy {accu_val:8.3f}')
    print('-' * 59)

52it [00:07, 10.07it/s]

| epoch   1 |    50/  372 batches | accuracy    0.492


101it [00:12, 10.04it/s]

| epoch   1 |   100/  372 batches | accuracy    0.514


153it [00:17, 10.83it/s]

...

| epoch  15 |   300/  372 batches | accuracy    0.541


352it [00:33, 10.79it/s]

| epoch  15 |   350/  372 batches | accuracy    0.564


372it [00:35, 10.43it/s]


-----------------------------------------------------------
| end of epoch  15 | time: 36.41s | valid accuracy    0.517
-----------------------------------------------------------


#### Test

In [9]:
print('Checking the results of test dataset.')
model.eval()
total_acc, total_count = 0, 0
with torch.no_grad():
    for idx, (text, label) in enumerate(test_dataloader):
        predicted_label = model(text, )
        loss = criterion(predicted_label, label)
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
accu_test =  total_acc/total_count
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.508


In [10]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text))
        text = pad_sequence([text], batch_first=True, padding_value=idx_pad)
        output = model(text)
        return output.argmax(1).item()

ex_text_str = "It was very bad movie"

model = model.to("cpu")
label_dict = {0:'neg', 1:'pos'}
print(f"This is a {label_dict.get(predict(ex_text_str, text_pipeline))} comment")

This is a neg comment
